http://www.patentsview.org/api/patent.html

In [276]:
import requests

In [277]:
fields = ["patent_abstract", "patent_date", "patent_num_cited_by_us_patents", "patent_title", 'patent_number']
fields.extend( [ 'cpc_subgroup_title', 'cpc_subgroup_id' ])
fields.extend( [ 'cited_patent_number' ] )
fields.extend(["inventor_first_name","inventor_last_name"])
fields.extend(["citedby_patent_number"])

FIELDS_str = '''[%s]''' % ', '.join( [ '''"%s"''' % f for f in fields ] )

URL = "http://www.patentsview.org/api/patents/query"

def query_patent( patent_number):
    
    query = '''{"patent_number":"%s"}''' % patent_number
    
    r = requests.get(URL, params={'q':query, 'f':FIELDS_str})
    r_json = r.json()
    data = r_json['patents'][0]
    
    # post-process
    citedby_patents = [ d['citedby_patent_number'] for d in data['citedby_patents'] if d['citedby_patent_number'] ] 
    data['citedby_patents'] = citedby_patents

    cited_patents = [ d['cited_patent_number'] for d in data['cited_patents'] if  d['cited_patent_number'] ] 
    data['cited_patents'] = cited_patents
    
    return data

In [278]:
import pickle

In [279]:
# -- cache --
#DATA = {}

DATA = pickle.load( open('DATA.pickle','rb') )

def get_patent( patent_number ):
    if patent_number in DATA:
        patent = DATA[patent_number]
    else:
        print(  'query patentview for %s'%patent_number )
        patent = query_patent( patent_number )
        DATA[ patent_number ] = patent
        
    return patent

In [280]:
pickle.dump( DATA, open('DATA.pickle','wb') )

In [281]:
r = get_patent( "6220251" )

In [282]:
print( r )

{'cited_patents': ['3955884', '4398800', '4640011', '4788768', '4809431', '4893406', '4930871', '4956915', '5046606', '5263754', '5357677', '5358297', '5370648', '5546658', '5775340', '5931167'], 'patent_date': '2001-04-24', 'patent_num_cited_by_us_patents': '23', 'citedby_patents': ['6381080', '6384990', '6523545', '6539632', '6553592', '6560874', '6683298', '6865812', '7131448', '7137356', '7263775', '7320180', '7464665', '7575335', '8096305', '8496013', 'D446352', 'D498333', 'D519677', 'D625884', 'D634066', 'D673733', 'D726374'], 'patent_title': 'Combination vision enhancement kit and nail clipper', 'patent_number': '6220251', 'cpcs': [{'cpc_subgroup_title': 'Manicuring or pedicuring implements-Nail files, e.g. manually operated', 'cpc_subgroup_id': 'A45D29/04'}, {'cpc_subgroup_title': 'Manicuring or pedicuring implements-Manicure or pedicure sets, e.g. combinations without case, etui, or the like', 'cpc_subgroup_id': 'A45D29/18'}, {'cpc_subgroup_title': 'Manicuring or pedicuring im

# graph 

finalement avec `pygraphviz`  
https://pygraphviz.github.io/documentation/latest/tutorial.html

In [283]:
import pygraphviz as pgv

In [284]:
# create graph
G = pgv.AGraph(strict=True, directed=True)

In [285]:
G.graph_attr['rankdir'] = 'LR' # orientation du graph lors du rendu: Left->Right

In [286]:
def add_node( patent, gen ):
    node_id = patent['patent_number']
    G.add_node( node_id, id=node_id, gen=gen )

### gen0

In [287]:
seed_number = "5307209"
#seed_number = "D470161"

seed_patent = get_patent( seed_number )

add_node( seed_patent, 0 )

In [288]:
print( G.string() )

strict digraph {
	graph [rankdir=LR];
	5307209	 [gen=0,
		id=5307209];
}



## gen1

In [289]:
# cited
for citation_number in seed_patent['cited_patents']:
    patent = get_patent(citation_number)
    
    add_node( patent, 1 )
    
# cited by
for citedby_number in seed_patent['citedby_patents']:
    patent = get_patent(citedby_number)
    
    add_node( patent, 1 )

In [290]:
# add edges

for node_id in G.nodes():
    
    patent_number = node_id
    
    patent = get_patent( patent_number )
    
    target = node_id
    for cited_number in patent['cited_patents']:
        if G.has_node( cited_number ):
            
            source = cited_number
            G.add_edge(source, target, id='-'.join([source, target]))

## Style

In [291]:
for e in G.edges( [seed_number] ): # selectionne les edges adjacents à seed_number
    e.attr['color'] = 'gray'

In [292]:
for node, deg in G.in_degree_iter(): 
    if deg == 0:
        node.attr['fillcolor'] = '#ffe6b3'
        node.attr['style'] = 'filled'

In [304]:
for node in G.nodes():
    patent = get_patent( node )
    node.attr['tooltip'] = '%s \n(%s) <b>test</b>'%(patent['patent_title'], patent['patent_date'])
    node.attr['label'] = patent['inventors'][0]['inventor_first_name'] +' '+ patent['inventors'][0]['inventor_last_name'] + '\n'+ patent['patent_date']
    node.attr['url'] = 'http://hello.world'

In [299]:
patent

{'cited_patents': ['3936156',
  '3955884',
  '4094598',
  '4190322',
  '4712870',
  '4819351',
  '5113289',
  '5204775',
  '5218482',
  '5245477',
  '5307209',
  '5309279',
  '5471347',
  '5615768',
  '5847883',
  '5926315',
  '6137639',
  '6233102',
  '7218463'],
 'citedby_patents': [],
 'cpcs': [{'cpc_subgroup_id': None, 'cpc_subgroup_title': None}],
 'inventors': [{'inventor_first_name': 'Paul F.',
   'inventor_last_name': 'Giampavolo'},
  {'inventor_first_name': 'Christine', 'inventor_last_name': 'Giampavolo'}],
 'patent_abstract': 'A magnification device for attachment to store merchandise shelving for magnifying merchandise labeling information including an optical magnification member; a bracket for attachment to merchandise shelving; the optical magnification member being attached to the bracket. A connector is preferably provided connecting the magnification member to the bracket and preferably allows the magnification member to be moved into one of a plurality of positions to

In [294]:
#G.node_attr['height'] = 1.5 # default 0.5

In [295]:
#n = G.get_node( seed_number )
#n.attr['image'] = "images/1996-US-5557849-A.png"
#n.attr['imagescale'] = True
#n.attr['fixedsize'] = True

# Save

In [306]:
G.draw('testgraph.svg', prog='dot')

In [305]:
filename_web = 'web/data/gen_graph_%s.dot' % seed_number
G.write(filename_web)

In [298]:
pickle.dump( DATA, open('DATA.pickle','wb') )